Script to download sequence data from NCBI for alignment, cropping and analysis of variation between species.

In [ ]:
# install package requests if you do not yet have it installed
# only need to run this once

import sys
!{sys.executable} -m pip install requests

In [ ]:
# read the list of species we are looking for from infile

from Bio import Entrez

infile = 'infiles/Suzanne_species_names.txt'              # or fish_species_names.txt or some other file
outfile_name = 'outfiles/raw_downloads/outfile.fas'    # CHANGE THIS or it will overwrite your last outfile ******************
locus = '16S'                                          # or cytb or COI, etc.
Entrez.email = ''                        # fill in your e-mail here (should be registered with NCBI)
API_KEY = ''       # use api_key if you have one (makes it faster to search at NCBI)

fish_sealprey = []
with open(infile) as f:                                 
    for line in f:
        fish_sealprey.append(line.strip())
f.close()

In [ ]:
from Bio import SeqIO

# Read all fish species into the list
fish_species = [line.strip() for line in open(infile)]

In [ ]:
# search NCBI for the locus for all species

acc_list = []
for fish_species in fish_sealprey:
    # example search term: ("Pterostichus agonus"[Organism] AND 16S[All Fields])
    species_term = '"' + fish_species + '"[Organism] AND ' + locus +'[All Fields]'
    search_handle = Entrez.esearch(db="nucleotide", term=species_term, idtype="acc", usehistory="y", api_key=API_KEY)
    record = Entrez.read(search_handle)
    search_handle.close()
    acc_list += (record["IdList"])

# print the first six accession numbers retrieved
print(acc_list[:6])

In [ ]:
# show how many sequence records were retrieved

print(len(acc_list))

In [ ]:
# obtain DNA sequence data for each accession number from NCBI

from Bio import Entrez
import requests

outfile = open(outfile_name, 'w')
for acc in acc_list:
    handle = Entrez.efetch(db="nucleotide", id=acc, rettype="fasta", retmode="text", api_key=API_KEY)
    # print output to file only if the sequence is not longer than mitochondrial size
    accession = handle.read()
    if len(accession) < 30000:
        outfile.write(accession.replace('N','').replace(' ',''))

print(outfile)
outfile.close()

In [ ]:
# produce command line for Clustalw2
# run the command line on a windows command prompt

from Bio.Align.Applications import ClustalwCommandline
cline = ClustalwCommandline("clustalw2", infile='outfile.fas')
print(cline)

now manually crop the alignment (*.aln) to the segment of interest<br>
also remove unalignable sequences<br>